# TenisIQ

Analisando o dataset: https://www.kaggle.com/datasets/gmadevs/atp-matches-dataset?resource=download

In these datasets there are individual csv files for ATP tournament from 2000 to 2017.

The numbers in the last columns are absolute values, using them you can calculate percentages.

* ace = absolute number of aces
* df = number of double faults
* svpt = total serve points
* 1stin = 1st serve in
* 1st won = points won on 1st serve
* 2ndwon = points won on 2nd serve
* SvGms = serve games
* bpSaved = break point saved
* bpFaced = break point faced

## **Importando os pacotes**

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from prettytable import PrettyTable
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


## **Importando os datasets**


Definindo cada planilha csv que se encontra dentro do zip no card do Trello e depois concatenando todos eles em um dataset só.

Além disso verificaremos se a concatenação foi feita da maneira correta com a soma das linas de todos os df, pondo todos eles em uma lista e utilizando o recurso shape.




In [2]:
#Definindo o dataframe:
data = pd.read_csv('dados_balanceados.csv')
df = pd.read_csv('dados_balanceados.csv')
dataIna = pd.read_csv('dados_balanceados.csv')

# Implementação do Random Forest

A ideia é fazer uma classificação categórica para determinar o vencedor.
Nesse caso, terão 3 colunas principais:
* Vencedor, Perderdor, e VencedorTarget.
* Os nomes dos vencedores/perdedores serão mapeados para um número, enquanto a VencedorTarget será a predita (dropada no treino).

Ideias posteriores:
* Aumentar o dataset com dados de bets, e ajustar o peso para prever melhor.
* Prever número de Aces
* Prever tempo do jogo, ou algo que aconteceria no jogo


Mas antes, vamos entender como ficaram os datasets:

In [3]:
def exibir_info_organizada(datasets, nomes):
    for df, nome in zip(datasets, nomes):
        print(f"Dataset: {nome}")
        print(f"{'Colunas':<20}: {', '.join(df.columns[:5])} ...")  # Mostra até 5 colunas para não poluir
        print(f"{'Número de linhas':<20}: {len(df)}")
        print(f"{'Primeiras linhas':<20}:\n{df.head().to_string(index=False)}\n")
        print("-" * 80)

# Supondo que os datasets 'data', 'df' e 'dataIna' já estejam carregados
# Exibindo as informações de forma organizada
datasets = [data, df, dataIna]
nomes = ['data', 'df', 'dataIna']

# Chamando a função
exibir_info_organizada(datasets, nomes)

Dataset: data
Colunas             : tourney_name, surface, draw_size, tourney_level, winner_seed ...
Número de linhas    : 30939
Primeiras linhas    :
 tourney_name  surface  draw_size  tourney_level  winner_seed  winner_name  winner_hand  winner_ht  winner_ioc  winner_age  winner_rank  winner_rank_points  loser_seed  loser_name  loser_hand  loser_ht  loser_ioc  loser_age  loser_rank  loser_rank_points  best_of  round
     0.964899 0.333333        0.0            0.0     0.053783          108          0.5   0.488889    0.326733    0.479505     0.000000                 0.0    0.000000    0.073276         0.5  0.377778   0.037037   0.316640         0.0                0.0      0.0  0.625
     0.964899 0.333333        0.0            0.0     0.053783          335          0.5   0.444444    0.158416    0.168995     0.133606                 0.0    0.049151    0.136315         0.5  0.222222   0.712963   0.310100         0.0                0.0      0.0  0.625
     0.964899 0.333333        0.0   

In [4]:
# Separar features e target
X = df.drop(['winner_name'], axis=1)
y = df['winner_name']

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Codificar os rótulos (se necessário)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [6]:
# Instanciar o modelo
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# Fazer previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy * 100:.2f}%')

# Outras métricas
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Acurácia: 89.53%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.54      0.61      0.57        33
           2       0.45      0.64      0.53        22
           3       0.74      0.92      0.82        38
           4       0.87      0.62      0.72        21
           5       0.62      0.67      0.64        12
           6       0.83      0.54      0.65        28
           7       1.00      1.00      1.00        18
           8       1.00      0.80      0.89        10
           9       0.75      0.55      0.63        11
          10       0.72      0.72      0.72        36
          11       1.00      0.97      0.98        30
          12       1.00      1.00      1.00        50
          13       0.93      0.97      0.95        66
          14       1.00      1.00      1.00        12
          15       0.89      0.97      0.93        34
          16       0.67      0.40      0.50        15
          

In [8]:
# Parâmetros otimizados para evitar sobrecarga
param_grid = {
    'n_estimators': [50, 100],            # Reduzi o número de valores testados
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']      # Limitei as opções de max_features
}

# Configuração de GridSearchCV com menos métricas e 3 folds
rf = RandomForestClassifier(random_state=42)
scoring = ['accuracy', 'f1']              # Usando apenas `accuracy` e `f1`
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring=scoring, refit='accuracy')

# Realiza o GridSearchCV nos dados de treino
grid_search.fit(X_train, y_train)

# Obtém os resultados e filtra colunas para evitar excessos de dados
results_df = pd.DataFrame(grid_search.cv_results_)
results_log = results_df[['params', 
                          'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy',
                          'mean_test_f1', 'std_test_f1']]

# Salva o log em um arquivo CSV
results_log.to_csv('log.csv', index=False)

print("Grid Search completo! Resultados salvos em 'log.csv'.")

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   4.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   4.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   4.2s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.8s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.9s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.9s


...

[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   6.3s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   6.3s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   6.2s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

Grid Search completo! Resultados salvos em 'log.csv'.


In [9]:
# Define os parâmetros para o DecisionTreeClassifier
param_grid_dt = {
    'criterion': ['gini', 'entropy'],         # Funções de medição de impureza
    'max_depth': [None, 10, 20, 30],          # Limite de profundidade da árvore
    'min_samples_split': [2, 5, 10],          # Número mínimo de amostras para dividir
    'min_samples_leaf': [1, 2, 4]             # Número mínimo de amostras em um nó folha
}

# Configura o GridSearchCV para o DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=3, n_jobs=-1, verbose=2, scoring=scoring, refit='accuracy')

# Realiza o GridSearchCV nos dados de treino
grid_search_dt.fit(X_train, y_train)

# Obtém os resultados em um DataFrame
results_df_dt = pd.DataFrame(grid_search_dt.cv_results_)

# Filtra colunas de interesse para o DecisionTreeClassifier
results_log_dt = results_df_dt[['params', 
                                'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy',
                                'mean_test_f1', 'std_test_f1']]

# Salva o log do DecisionTreeClassifier em um arquivo separado
results_log_dt.to_csv('log_decision_tree.csv', index=False)

print("Grid Search para Decision Tree completo! Resultados salvos em 'log_decision_tree.csv'.")

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.2s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.3s


...

[CV] END criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=2; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=2; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=2; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=5; total time=   0.4s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=5; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=5; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=10; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=10; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=2, min_samples_split=10; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2; total time=   0.4s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=5; total time=   0.3s


/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/home/jupyter-jphuser04/projects/word2vec/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1239, in f1_score
    return fbeta_score(
  File "/home/jupy

[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=5; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=5; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=10; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=10; total time=   0.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=10; total time=   0.4s
Grid Search para Decision Tree completo! Resultados salvos em 'log_decision_tree.csv'.
